In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import svd
import backtrader as bt
from tqdm.notebook import tqdm # Используем tqdm.notebook для Jupyter

# Убедимся, что графики Backtrader отображаются в Jupyter (если нужно)
# %matplotlib inline # или %matplotlib widget, но widget может конфликтовать с tqdm в некоторых средах

# ---------- SSA helpers (без изменений) ----------------------------------
def ssa_decompose(series, window):
    # K = N - L + 1 (number of columns in trajectory matrix)
    # L = window (number of rows in trajectory matrix)
    # N = len(series)
    # Исправлено: X должен иметь K столбцов, а не K строк.
    # Каждая строка X - это сдвинутый отрезок серии длины window.
    # Количество таких отрезков (столбцов в моей первоначальной логике X) K = len(series) - window + 1
    # Траекторная матрица X должна быть L x K
    # X = np.array([series[i : i + window] for i in range(len(series) - window + 1)])
    # Это формирует матрицу K x window. Нам нужна L x K.
    # Поэтому нужно либо транспонировать, либо формировать столбцами.
    # Стандартный подход:
    K = len(series) - window + 1 # Number of columns
    # X = np.column_stack([series[i:i+window] for i in range(K)]) # Это L x K
    
    # Оригинальный код формирует X как window x K, где K = len(series) - window + 1
    # X = np.vstack([series[i:i+K] for i in range(window)]) # Это неверно, если следовать классическому SSA
    # Если series[i:i+K] - это i-я строка, то это L строк, K столбцов.
    # K в данном контексте (series[i:i+K]) это длина каждого такого "лага".
    # В классическом SSA, X - это матрица L x K, где L - window, K = N - L + 1
    # X_ij = series[i+j-2] for i=1..L, j=1..K
    # В Python:
    # X = np.array([series[j:j+window] for j in range(len(series) - window + 1)]).T
    # Это даст L x K.

    # Давай придерживаться логики ИЗНАЧАЛЬНОГО кода автора, даже если она отличается от "канона"
    # В оригинальном коде X получается window x (len(series) - window + 1)
    # U будет window x min(window, K)
    # s будет min(window, K)
    # Vt будет min(window, K) x K
    
    # В оригинальном коде:
    # K = len(series) - window + 1
    # X = np.vstack([series[i:i+K] for i in range(window)])
    # Это создает матрицу `window` строк и `K` столбцов.
    # X[j] = series[j : j + K]
    # Размер X: (window, K)
    # U: (window, min(window,K))
    # s: (min(window,K),)
    # Vt: (min(window,K), K)
    # Это немного нестандартное формирование траекторной матрицы, но будем следовать ему,
    # так как он работал в исходном коде. Классически, Xij = s_{i+j-2}.
    # Матрица X должна быть L x K, где L - длина окна, K - количество векторов-столбцов (N-L+1)
    # X_j = (s_j, s_{j+1}, ..., s_{j+L-1})^T.  X = [X_0, X_1, ..., X_{K-1}]
    
    N_series = len(series)
    if N_series < window:
        # Недостаточно данных для формирования траекторной матрицы
        # Можно вернуть None или пустые массивы, чтобы вызывающий код это обработал
        # print(f"Warning: series length {N_series} is less than window {window}. SSA not possible.")
        return None, None, None, window


    # Классическая траекторная матрица (L x K)
    # L = window
    # K_traj = N_series - window + 1
    # X = np.zeros((window, K_traj))
    # for i in range(K_traj):
    #     X[:, i] = series[i : i + window]
    
    # Код из вопроса использует другую форму траекторной матрицы, давайте её сохраним для консистентности:
    # Это создаст матрицу `window` строк и `K_cols` столбцов
    # где `K_cols` это `len(series) - window + 1`
    # X[j] будет рядом `series[j:j+K_cols]`
    # Таким образом, `X` будет `window` на `K_cols`.
    # U будет `window` на `min(window, K_cols)`
    # s будет `min(window, K_cols)`
    # Vt будет `min(window, K_cols)` на `K_cols`
    
    K_cols = N_series - window + 1 # Количество столбцов в X, которые являются лаговыми векторами
                                 # И это же длина каждого вектора-строки в оригинальной логике!
    if K_cols <= 0: # Это означает N_series < window
        return None, None, None, window
        
    # Оригинальная логика траекторной матрицы из вопроса:
    # X = np.vstack([series[i:i+K_cols] for i in range(window)])
    # Это означает, что i-я строка X это series[i], series[i+1], ..., series[i+K_cols-1]
    # Это делает X матрицей window x K_cols
    # Это означает, что каждая строка X — это сдвинутый временной ряд длиной K_cols.
    # Это не стандартная траекторная матрица для SSA.
    # Стандартная: столбцы являются сдвинутыми временными рядами длиной window.
    # X_j = (y_j, y_{j+1}, ..., y_{j+window-1})^T
    # X = [X_0, X_1, ..., X_{N-window}]
    # Размер X: window x (N_series - window + 1)

    # Давайте реализуем стандартную траекторную матрицу, так как реконструкция предполагает ее
    L_ssa = window
    K_ssa = N_series - L_ssa + 1
    if K_ssa <= 0: # N_series < L_ssa
        return None, None, None, L_ssa
        
    X = np.array([series[i:i+L_ssa] for i in range(K_ssa)]).T
    # Теперь X имеет размеры L_ssa x K_ssa
    
    try:
        U, s, Vt = svd(X, full_matrices=False)
    except np.linalg.LinAlgError:
        # print(f"SVD did not converge for series segment of length {N_series}, window {L_ssa}")
        return None, None, None, L_ssa
        
    return U, s, Vt, L_ssa # L_ssa это window

def reconstruct_trend(U, s, Vt, window, idx):
    if U is None: # Добавлено из-за возможного сбоя в ssa_decompose
        return np.array([])

    if isinstance(idx, int):
        idx = [idx]
    
    # U: L x r, s: r, Vt: r x K
    # r = min(L, K)
    # X_approx = U[:, idx] @ np.diag(s[idx]) @ Vt[idx, :]
    # В numpy:
    Xn = (U[:, idx] * s[idx]) @ Vt[idx, :] # Это правильно, если s - одномерный массив

    # Диагонализация для усреднения (анти-диагонализация)
    # Xn имеет размеры L x K (window x (N - window + 1))
    # N_recon = L + K - 1 = window + (N_series - window + 1) - 1 = N_series
    
    L_ssa, K_ssa = Xn.shape # L_ssa = window
    N_recon = L_ssa + K_ssa - 1
    
    recon = np.zeros(N_recon)
    counts = np.zeros(N_recon)

    # Процесс усреднения по анти-диагоналям
    for i in range(L_ssa): # по строкам Xn (компоненты элементарных матриц)
        for j in range(K_ssa): # по столбцам Xn
            recon[i+j] += Xn[i,j]
            counts[i+j] += 1
    
    # Избегаем деления на ноль, если counts где-то нулевые (не должно быть при правильной логике)
    counts[counts == 0] = 1 
    return recon / counts


# ---------- load data (без изменений) ------------------------------------
try:
    df = pd.read_csv('../csv/MSFT_M1_202402291729_202503272108.csv', sep='\t')
    df = df[:1000]
except FileNotFoundError:
    print("Файл данных не найден. Пожалуйста, проверьте путь '../csv/MSFT_M1_202402291729_202503272108.csv'")
    exit()
    
df['Datetime'] = pd.to_datetime(df['<DATE>'] + ' ' + df['<TIME>'])
df.set_index('Datetime', inplace=True)
df.drop(columns=['<DATE>', '<TIME>'], inplace=True)
df.rename(columns={'<OPEN>': 'Open', '<HIGH>': 'High', '<LOW>': 'Low', '<CLOSE>': 'Close', '<TICKVOL>': 'TickVol', '<VOL>': 'Volume'}, inplace=True)

# ---------- parameters (без изменений) -----------------------------------
series_all = df['Close'].to_numpy()

window      = 60
k_trend     = list(range(5)) # Используем первые 5 компонент для тренда
initial_len = 300 # Минимальная длина истории для первого расчета SSA
step        = 1    # Шаг для обновления расчета SSA
# n_updates   = 1000 # Ограничим для скорости, или до конца данных
# Определим n_updates так, чтобы обработать все данные после initial_len
if len(series_all) > initial_len:
    n_updates = (len(series_all) - initial_len) // step + 1
else:
    n_updates = 0
    print("Warning: Длина series_all меньше или равна initial_len. 'Черная линия' не будет построена.")


# ---------- build "black line" (SSA Trend) -------------------------------
print("Построение 'черной линии' (SSA Trend)...")
ssa_trend_values = []
ssa_trend_indices = [] # Индексы в исходном series_all

# Используем tqdm для отслеживания прогресса
for i in tqdm(range(n_updates)):
    end_idx = initial_len + i * step
    if end_idx > len(series_all):
        break
    
    current_segment = series_all[:end_idx]
    
    # Убедимся, что сегмент достаточно длинный для SSA
    if len(current_segment) < window:
        # print(f"Пропуск итерации {i}: длина сегмента {len(current_segment)} < окна {window}")
        continue # Пропускаем, если данных слишком мало

    U, s, Vt, win_actual = ssa_decompose(current_segment, window)
    
    if U is None or s is None or Vt is None: # Если SSA не удалось
        # print(f"Пропуск итерации {i} в {end_idx-1}: SSA не удалось.")
        # Можно попробовать использовать предыдущее значение тренда или пропустить точку
        if ssa_trend_values: # если уже есть значения
             ssa_trend_values.append(ssa_trend_values[-1]) # повторяем последнее значение
        else: # если это первая неудачная попытка
             ssa_trend_values.append(np.nan) # или current_segment[-1] если нужен fallback на цену
        ssa_trend_indices.append(end_idx - 1)
        continue

    # Убедимся, что k_trend не выходит за пределы количества сингулярных чисел
    valid_k_trend = [k_val for k_val in k_trend if k_val < len(s)]
    if not valid_k_trend:
        # print(f"Пропуск итерации {i} в {end_idx-1}: нет валидных k_trend (len(s)={len(s)}).")
        if ssa_trend_values:
             ssa_trend_values.append(ssa_trend_values[-1])
        else:
             ssa_trend_values.append(np.nan)
        ssa_trend_indices.append(end_idx - 1)
        continue
        
    reconstructed = reconstruct_trend(U, s, Vt, win_actual, valid_k_trend)
    
    if len(reconstructed) > 0:
        ssa_trend_values.append(reconstructed[-1])
        ssa_trend_indices.append(end_idx - 1) # Индекс соответствует последнему значению в current_segment
    else:
        # print(f"Пропуск итерации {i} в {end_idx-1}: реконструкция не дала результатов.")
        if ssa_trend_values:
             ssa_trend_values.append(ssa_trend_values[-1])
        else:
             ssa_trend_values.append(np.nan)
        ssa_trend_indices.append(end_idx - 1)

    # if i % (n_updates // 10 if n_updates > 10 else 1) == 0 and len(reconstructed) > 0 : # Печать прогресса
    #     print(f"i = {i}, end_idx = {end_idx}, trend_val = {reconstructed[-1]:.2f}", flush=True)

# Создаем Pandas Series для SSA Trend с правильным индексом Datetime
# ssa_trend_indices - это числовые индексы из series_all (numpy array)
# Нам нужны соответствующие им Datetime индексы из df
datetime_indices_for_ssa = df.index[ssa_trend_indices]
ssa_trend_series = pd.Series(ssa_trend_values, index=datetime_indices_for_ssa, name='SSATrend')

# Добавляем SSATrend в основной DataFrame
df_for_backtrader = df.copy()
df_for_backtrader = df_for_backtrader.join(ssa_trend_series) # Присоединяем по индексу Datetime

# Удаляем строки, где SSATrend еще не рассчитан (NaN в начале)
# или где SSA мог дать сбой
df_for_backtrader.dropna(subset=['SSATrend'], inplace=True) 

# Также удалим строки, где Volume может быть 0 или NaN, если используется для фильтрации
df_for_backtrader = df_for_backtrader[df_for_backtrader['Volume'] > 0]


# Проверка, что данные остались
if df_for_backtrader.empty:
    print("После обработки DataFrame для Backtrader пуст. Проверьте параметры SSA и данные.")
    exit()

print(f"Размер DataFrame для Backtrader: {df_for_backtrader.shape}")
print(f"Первые строки SSATrend:\n{df_for_backtrader['SSATrend'].head()}")
print(f"Последние строки SSATrend:\n{df_for_backtrader['SSATrend'].tail()}")


# ---------- Backtrader Strategy ------------------------------------------
class SSAReversalStrategy(bt.Strategy):
    params = (
        ('ssa_trend_line', 'SSATrend'), # Имя столбца с трендом SSA
        ('debug', False),
    )

    def __init__(self):
        # Убедимся, что линия SSATrend существует в данных
        if self.p.ssa_trend_line not in self.datas[0].lines.getlinealiases():
            raise ValueError(f"Линия '{self.p.ssa_trend_line}' не найдена в данных. Доступные линии: {self.datas[0].lines.getlinealiases()}")

        self.ssa_trend = self.datas[0].lines.getlineobject(self.p.ssa_trend_line)
        
        # Рассчитываем наклон SSATrend
        # self.ssa_slope = self.ssa_trend - self.ssa_trend(-1) # Наклон за 1 период
        # Используем более длинный наклон, чтобы сгладить шум, если нужно, но пока оставим 1 период
        self.ssa_slope = bt.indicators.ROC(self.ssa_trend, period=1, plot=False) # Эквивалентно (trend[0] - trend[-1])/trend[-1] * 100
                                                                                 # Нам нужен просто (trend[0] - trend[-1])
        
        # Для простого вычитания, если ROC не подходит:
        # self.ssa_slope = bt.Indicator() # Пустышка, чтобы Backtrader управлял длиной
        # self.ssa_slope.lines.slope = self.ssa_trend - self.ssa_trend(-1)
        # self.ssa_slope.plotinfo.plot = False # Не рисуем отдельно, если не нужно

        # Или еще проще, если индикатор не нужен глобально:
        # self.current_slope = 0
        # self.previous_slope = 0
        # (и обновлять в next)

        self.order = None
        self.trade_count = 0

    def log(self, txt, dt=None):
        if self.p.debug:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()} {txt}')

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return # Ничего не делать для этих статусов

        if order.status in [order.Completed]:
            if order.isbuy():
                self.log(f'BUY EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
            elif order.issell():
                self.log(f'SELL EXECUTED, Price: {order.executed.price:.2f}, Cost: {order.executed.value:.2f}, Comm: {order.executed.comm:.2f}')
            self.bar_executed = len(self)
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')
        
        self.order = None # Сбрасываем ордер после его обработки

    def notify_trade(self, trade):
        if not trade.isclosed:
            return
        self.log(f'OPERATION PROFIT, GROSS {trade.pnl:.2f}, NET {trade.pnlcomm:.2f}')
        self.trade_count +=1


    def next(self):
        # Ждем, пока сформируется достаточно данных для наклона
        # self.ssa_slope требует 2 значения ssa_trend
        # ssa_trend уже есть в данных, поэтому дополнительных ожиданий не нужно, кроме стандартного minperiod индикатора
        if len(self.ssa_trend) < 2 : # Нужно хотя бы 2 точки для расчета первого наклона
             return

        current_slope = self.ssa_trend[0] - self.ssa_trend[-1]
        previous_slope = self.ssa_trend[-1] - self.ssa_trend[-2] # Наклон на предыдущем шаге

        if self.order: # Если есть активный ордер, ничего не делаем
            return

        # Логика для первой сделки
        if not self.position:
            if current_slope > 0:
                self.log(f'INITIAL BUY: Current Slope {current_slope:.4f} > 0')
                self.order = self.buy()
            elif current_slope < 0:
                self.log(f'INITIAL SELL: Current Slope {current_slope:.4f} < 0')
                self.order = self.sell()
            return # Выходим после первой сделки

        # Логика разворота
        # Если знак наклона изменился
        # current_slope > 0 and previous_slope < 0  (разворот вверх)
        # current_slope < 0 and previous_slope > 0  (разворот вниз)
        if np.sign(current_slope) != np.sign(previous_slope) and previous_slope != 0: # Избегаем реакции на нулевой наклон
            if self.position.size > 0: # Если в длинной позиции
                if current_slope < 0: # и наклон стал отрицательным
                    self.log(f'CLOSING LONG and GOING SHORT: Current Slope {current_slope:.4f}, Prev Slope {previous_slope:.4f}')
                    self.order = self.close() # Сначала закрываем текущую
                    self.order = self.sell()  # Затем открываем противоположную (можно сделать через один ордер sell, если stake фиксированный)
            elif self.position.size < 0: # Если в короткой позиции
                if current_slope > 0: # и наклон стал положительным
                    self.log(f'CLOSING SHORT and GOING LONG: Current Slope {current_slope:.4f}, Prev Slope {previous_slope:.4f}')
                    self.order = self.close()
                    self.order = self.buy()
    
    def stop(self):
        self.log(f'(SSA Window {window}) Ending Value {self.broker.getvalue():.2f}, Trades: {self.trade_count}')


# ---------- Backtrader Execution -----------------------------------------
if df_for_backtrader.empty or len(df_for_backtrader) < window + 2: # +2 для возможности рассчитать два наклона
    print("Недостаточно данных для запуска Backtrader после предобработки.")
else:
    cerebro = bt.Cerebro()

    # Создаем кастомный PandasData фид
    # Необходимо указать, какие столбцы чему соответствуют
    # И добавить наш SSATrend как дополнительную линию
    class PandasDataWithSSATrend(bt.feeds.PandasData):
        lines = ('SSATrend',) # Определяем нашу дополнительную линию
        params = (
            ('datetime', None), # Используем индекс DataFrame
            ('open', 'Open'),
            ('high', 'High'),
            ('low', 'Low'),
            ('close', 'Close'),
            ('volume', 'Volume'),
            ('openinterest', None), # Нет данных о OI
            ('SSATrend', 'SSATrend'), # Указываем столбец для нашей линии
        )

    data_feed = PandasDataWithSSATrend(dataname=df_for_backtrader)
    cerebro.adddata(data_feed)

    cerebro.addstrategy(SSAReversalStrategy, debug=False) # Включите debug=True для подробных логов

    # Начальный капитал
    cerebro.broker.setcash(100000.0)
    # Размер позиции (например, 100 акций)
    cerebro.addsizer(bt.sizers.FixedSize, stake=100)
    # Комиссия (0.1%)
    cerebro.broker.setcommission(commission=0.001)

    # Анализаторы
    cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='sharpe_ratio')
    cerebro.addanalyzer(bt.analyzers.AnnualReturn, _name='annual_return')
    cerebro.addanalyzer(bt.analyzers.DrawDown, _name='drawdown')
    cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    results = cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # Печать результатов анализаторов
    strat = results[0]
    print('\n--- Analyzers Results ---')
    print(f"Sharpe Ratio: {strat.analyzers.sharpe_ratio.get_analysis()['sharperatio']:.2f}")
    # print(f"Annual Return: {strat.analyzers.annual_return.get_analysis()}") # AnnualReturn может быть словарем
    print(f"Max Drawdown: {strat.analyzers.drawdown.get_analysis()['max']['drawdown']:.2f}%")
    
    trade_analysis = strat.analyzers.trade_analyzer.get_analysis()
    if trade_analysis:
      print(f"Total Trades: {trade_analysis.total.total}")
      if trade_analysis.total.total > 0:
          print(f"Winning Trades: {trade_analysis.won.total}")
          print(f"Losing Trades: {trade_analysis.lost.total}")
          print(f"Win Rate: {trade_analysis.won.total / trade_analysis.total.total * 100:.2f}%")
          print(f"Average Win: {trade_analysis.won.pnl.average:.2f}")
          print(f"Average Loss: {trade_analysis.lost.pnl.average:.2f}")
          print(f"Profit Factor: {abs(trade_analysis.won.pnl.total / trade_analysis.lost.pnl.total if trade_analysis.lost.pnl.total != 0 else 'inf'):.2f}")


    # Построение графика
    # %matplotlib widget # Для интерактивных графиков в Jupyter Lab/Notebook
    # или
    # %matplotlib inline # Для статических графиков
    # plt.style.use('seaborn-v0_8-darkgrid') # Попробуем другой стиль для лучшей читаемости
    
    # Уменьшаем размер фигуры, чтобы легенда не перекрывала
    fig = cerebro.plot(figsize=(12, 8), style='candlestick', barup='green', bardown='red')[0][0] # Получаем объект фигуры
    
    # Можно настроить параметры графика, если нужно
    # fig.suptitle('Стратегия разворота на SSA', fontsize=16)
    # ax = fig.get_axes()[0] # Получаем оси главного графика
    # ax.legend(loc='upper left')
    
    plt.show()